<a href="https://colab.research.google.com/github/sm-lee-1/Caba_nlp/blob/main/2%EC%A3%BC%EC%B0%A8_2_preprocessing_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BPE 알고리즘을 이해하고 텍스트 파일을 tfrecord로 저장하기

강의자 : 안태경

지난 과제로 만들었던 텍스트 데이터를 BPE 알고리즘을 통해 tokenizing하고 tfrecord로 저장해 봅시다.



---



### BPE 알고리즘 예제

출처 : 딥 러닝을 이용한 자연어 처리 입문 https://wikidocs.net/22592

논문 : https://arxiv.org/pdf/1508.07909.pdf

In [ ]:
import re, collections
from IPython.display import display, Markdown, Latex

In [ ]:
# BPE을 몇 회 수행할 것인지 설정
num_merges = 10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# </w> 는 단어의 맨 끝을 표시하는 문자
# 각 단어는 글자(character) 단위로 분리
# {훈련 데이터에 있는 단어 : 등장 빈도 수 , ....}

# 특정 데이터에 해당 단어들이 아래와 같은 빈도만큼 등장했다고 가정
dictionary = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

print(dictionary)

{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 6, 'w i d e s t </w>': 3}


In [ ]:
# BPE 코드

def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    pair_dict = dict(pairs)
    pair_sort = sorted(pair_dict.items(), key = lambda item : item[1], reverse=True)
    print('현재 pair들의 빈도수 :', pair_sort)
    return pairs

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

In [ ]:
bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1)))
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print("new merge: {}".format(best))
    print("dictionary: {}".format(dictionary))

### Iteration 1

현재 pair들의 빈도수 : [(('e', 's'), 9), (('s', 't'), 9), (('t', '</w>'), 9), (('w', 'e'), 8), (('l', 'o'), 7), (('o', 'w'), 7), (('n', 'e'), 6), (('e', 'w'), 6), (('w', '</w>'), 5), (('w', 'i'), 3), (('i', 'd'), 3), (('d', 'e'), 3), (('e', 'r'), 2), (('r', '</w>'), 2)]
new merge: ('e', 's')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

현재 pair들의 빈도수 : [(('es', 't'), 9), (('t', '</w>'), 9), (('l', 'o'), 7), (('o', 'w'), 7), (('n', 'e'), 6), (('e', 'w'), 6), (('w', 'es'), 6), (('w', '</w>'), 5), (('w', 'i'), 3), (('i', 'd'), 3), (('d', 'es'), 3), (('w', 'e'), 2), (('e', 'r'), 2), (('r', '</w>'), 2)]
new merge: ('es', 't')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

현재 pair들의 빈도수 : [(('new', 'est</w>'), 6), (('low', '</w>'), 5), (('w', 'i'), 3), (('i', 'd'), 3), (('d', 'est</w>'), 3), (('low', 'e'), 2), (('e', 'r'), 2), (('r', '</w>'), 2)]
new merge: ('new', 'est</w>')
dictionary: {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

현재 pair들의 빈도수 : [(('low', '</w>'), 5), (('w', 'i'), 3), (('i', 'd'), 3), (('d', 'est</w>'), 3), (('low', 'e'), 2), (('e', 'r'), 2), (('r', '</w>'), 2)]
new merge: ('low', '</w>')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

현재 pair들의 빈도수 : [(('w', 'i'), 3), (('i', 'd'), 3), (('d', 'est</w>'), 3), (('low', 'e'), 2), (('e', 'r'), 2), (('r', '</w>'), 2)]
new merge: ('w', 'i')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


e와 s의 쌍은 초기 단어 집합에서 총 9회 등장했습니다. 그렇기 때문에 es로 통합됩니다. 그 다음으로는 es와 t의 쌍을, 그 다음으로는 est와 </w>의 쌍을 통합시킵니다. 빈도수가 가장 높은 순서대로 통합하는 이 과정을 총 num_merges회 반복한 것입니다.

In [ ]:
# merge 했던 기록 출력
print(bpe_codes)

{('e', 's'): 0, ('es', 't'): 1, ('est', '</w>'): 2, ('l', 'o'): 3, ('lo', 'w'): 4, ('n', 'e'): 5, ('ne', 'w'): 6, ('new', 'est</w>'): 7, ('low', '</w>'): 8, ('w', 'i'): 9}


이 사전은 새로운 단어가 등장하였을 때, 현재 가지고 있는 서브워드 단어 집합에 기반하여 분리하는 작업에 참고할 수 있습니다.



---
## Preprocessing 진행하기 예시

In [4]:
!pwd

/content


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#### 1. 본인의 Github에 ALBERT 깃을 포크합니다.

* https://github.com/google-research/ALBERT

* 화면 오른쪽 위의 Fork 버튼을 눌러주세요.

#### 2. 자신의 깃헙에서 albert를 clone하고 필요한 모듈을 설치합니다.


In [2]:
! git clone https://github.com/sm-lee-1/albert.git
% pip install -r albert/requirements.txt

Cloning into 'albert'...
remote: Enumerating objects: 356, done.
remote: Total 356 (delta 0), reused 0 (delta 0), pack-reused 356
Receiving objects: 100% (356/356), 235.95 KiB | 5.13 MiB/s, done.
Resolving deltas: 100% (233/233), done.
     |████████████████████████████████| 110.5MB 46kB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 1.2MB 41.4MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 512kB 35.6MB/s 
     |████████████████████████████████| 3.8MB 47.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=f7075ecf5382a1639c5e14714c3a644343877e039a0b106737716aec3e483baf
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorf

#### 3. 준비한 데이터로 vocab(단어사전)을 만들어 줍니다.

* 주의 : vocab size는 5000을 넘기지 않도록 해주세요. Colab 환경 상 그 이상으로는 훈련이 어렵습니다.
* 3000에서 5000 사이로 설정해주시면 됩니다. 참고로 원래 vocab을 만들 때에는 사이즈를 더 크게(30000정도로) 설정합니다.

```import sentencepiece as spm```   
```spm.SentencePieceTrainer.Train('--unk_piece=[UNK] --pad_piece=<pad> --bos_id=-1 eos_id=-1 --pad_id=0 --unk_id=1 --user_defined_symbols=[CLS],[SEP],[MASK] --input={큰 텍스트파일 경로} --model_prefix={보캡 파일 이름} --model_type=bpe --vocab_size={보캡사이즈}')``` 

In [5]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--unk_piece=[UNK] --pad_piece=<pad> --bos_id=-1 eos_id=-1 --pad_id=0 --unk_id=1 \
--user_defined_symbols=[CLS],[SEP],[MASK] --input=/content/gdrive/MyDrive/Caba2012_colab/text_crawling/output.txt \
--model_prefix=/content/gdrive/MyDrive/Caba2012_colab/text_crawling/vocab_sample --model_type=bpe --vocab_size=3000')

#### 4. ALBERT의 create_pretraining_data.py를 실행하여 tfrecord를 만들어 줍니다.

* 해당 코드는 tokenization.py를 불러와 tokenizing을 하고 tfrecord로 저장해주는 기능을 합니다.

* tokenization.py의 FullTokenizer 클래스가 spm에서 훈련된 모델을 불러와 BPE 알고리즘이 실행되는 방식입니다. 

  (tokenization.py line 29, line 180 참조)

```! python -m albert.create_pretraining_data  --input_file={txt 파일들 경로}  --output_file={tfrecord 경로} --vocab_file={보캡 파일 경로} --spm_model_file={모델 파일 경로}``` 


In [8]:
! python -m albert.create_pretraining_data --input_file=/content/gdrive/MyDrive/Caba2012_colab/text_crawling/output.txt --output_file=/content/gdrive/MyDrive/Caba2012_colab/text_crawling/output/output.tfrecord --vocab_file=/content/gdrive/MyDrive/Caba2012_colab/text_crawling/vocab_sample.vocab --spm_model_file=/content/gdrive/MyDrive/Caba2012_colab/text_crawling/vocab_sample.model

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
INFO:tensorflow:using sentence piece tokenzier.
I0407 05:12:56.988555 139889755977600 tokenization.py:237] using sentence piece tokenzier.
INFO:tensorflow:using sentence piece tokenzier.
I0407 05:12:56.989201 139889755977600 tokenization.py:237] using sentence piece tokenzier.
INFO:tensorflow:using sentence piece tokenzier.
I0407 05:12:56.989989 139889755977600 tokenization.py:237] using sentence piece tokenzier.
INFO:tensorflow:using sentence piece tokenzier.
I0407 05:12:56.990698 139889755977600 tokenization.py:237] using sentence piece tokenzier.
INFO:tensorflow:using sentence piece tokenzier.
I0407 05:12:56.991389 139889755977600 tokenization.py:237] using sentence piece tokenzier.
INFO:tensorflow:using sentence piece tokenzier.
I0407 05:12:56.992674 139889755977600 tokenization.py:237] using sentence piece tokenzier.
INFO:tensorflow:using sentence piece tokenzier.
I0407 05:12:56.993118 139889755977600 tokenization.py:237] using sentence piece to

##### ** 주의할 점 : 예시에서 사용한 파일은 아주 작은 텍스트 파일이지만, 과제로 크롤링하여 정제한 결과물은 큰 텍스트 파일이기 때문에, vocab을 만들 때 사용한 큰 텍스트 파일을 여러 파일로 작게 쪼개서 여러 개의 tfrecord로 만들어 주어야 합니다. 아래 코드를 참고하셔서 과제를 진행해 주세요.

```! split -d -l 1000 /content/drive/MyDrive/{큰 텍스트 파일 경로}/{텍스트파일이름}.txt /content/drive/MyDrive/{출력 경로}/small_```

위의 코드를 돌리면 출력 경로 밑에 small_00, small_01 ... 의 파일이 차례대로 생깁니다

```! for file in {이 부분은 채워 보세요} ; do python -m albert.create_pretraining_data --input_file=$file --output_file=${file}.tfrecord --vocab_file=/content/drive/MyDrive/{.vocab경로} --spm_model_file=/content/drive/MyDrive/{.model경로}; done```

#### 5. tfrecord가 잘 만들어졌는지 확인해 봅니다.

In [9]:
from albert.tokenization import FullTokenizer
import tensorflow

tokenizer = FullTokenizer(vocab_file=None, spm_model_file='/content/gdrive/MyDrive/Caba2012_colab/text_crawling/vocab_sample.model')

i = 0
for example in tensorflow.python_io.tf_record_iterator("/content/gdrive/MyDrive/Caba2012_colab/text_crawling/output/output.tfrecord"):

    paragraph = (list(tensorflow.train.Example.FromString(example).features.feature['input_ids'].int64_list.value))
    para_tokens = (tokenizer.convert_ids_to_tokens(paragraph))
    print('/'.join(para_tokens))
    print('===========================')
    i += 1
    if i > 5:
      break

INFO:tensorflow:loading sentence piece model
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:tensorflow:using sentence piece tokenzier.
[CLS]/▁결국/[MASK]/[MASK]/[MASK]/르/제/트로/▁교체/됐/고/▁양현종은/[MASK]/이닝도/▁버/티지/▁못하고/▁강판됐다/./▁그/나/마/▁양현종/▁이후/▁올라/온/▁불/펜/이/▁실/점/없이/▁이/릿/을/▁마/치/며/▁양현종의/▁추가/[MASK]/은/▁없었다/./[SEP]/▁양현종은/▁기/존/▁시범경기/▁평균/자책/[MASK]/이/▁3/.8/6/에서/▁5/.4/0/이/▁됐/고/▁메이저리그/▁생존/이/▁쉽/지/▁않/을/▁것으로/▁보인다/./[SEP]/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>/<pad>



---

### 이번 주 수업 과제

지난 강의에서 크롤링과 텍스트 정제를 통해 얻은 깨끗한 텍스트 데이터를 가지 preprocessing을 완료하여 작성한 코드와 결과물(tfrecord 파일들)을 구글 드라이브에 업로드해 주세요.

과제에서 크롤링으로 수집한 뒤에 정제한 데이터는 큰 데이터이기 때문에, 위의 예시와 다르게 여러 개의 텍스트 파일로 쪼개서 tfrecord를 만들어 주셔야 합니다!

Github 계정이 아직 없으신 분들은 계정을 만들고, 위와 같은 방법으로 albert코드를 본인의 github 계정으로 포크하셔서 과제를 수행해 주시면 됩니다.

과제는 금요일 정오(12시)까지 제출해 주시면 됩니다.
